In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler

### JKP processing

In [ ]:

# CSV読み込み
jkp_long = pd.read_csv("[usa]_[all_factors]_[monthly]_[vw_cap].csv", parse_dates=["date"])

# ピボットで wide 形式に変換（date × factor_name → ret）
jkp_wide = jkp_long.pivot(index="date", columns="name", values="ret")

jkp_wide = jkp_wide[jkp_wide.index >= '2005-01-01']


missing_ratio = jkp_wide.isnull().mean()
jkp_filtered = jkp_wide.loc[:, missing_ratio < 0.3]
n_dropped = (missing_ratio >= 0.3).sum()
print('The number of dropped factors due to high missing ratio:', n_dropped)


def winsorize_df(df, lower_pct=0.01, upper_pct=0.99):
    df_winsorized = df.copy()
    for col in df.columns:
        lower = df[col].quantile(lower_pct)
        upper = df[col].quantile(upper_pct)
        df_winsorized[col] = np.clip(df[col], lower, upper)
    return df_winsorized

jkp_winsorized = winsorize_df(jkp_filtered, lower_pct=0.01, upper_pct=0.99)


variances = jkp_winsorized.var()
low_var_cols = variances[variances < 1e-5].index
jkp_var_filtered = jkp_winsorized.drop(columns=low_var_cols)
print('The dropped factors due to low variance:', len(low_var_cols))


corr_matrix = jkp_var_filtered.corr().abs()
upper_triangle = np.triu(corr_matrix, k=1)
upper_df = pd.DataFrame(upper_triangle, index=corr_matrix.index, columns=corr_matrix.columns)
threshold = 0.9
to_drop = [column for column in upper_df.columns if any(upper_df[column] > threshold)]
jkp_corr_filtered = jkp_var_filtered.drop(columns=to_drop)

print('The number of dropped factors due to high correlation:', len(to_drop))


selected_factors = [
    'beta_60m', 'bidaskhl_21d', 'dolvol_var_126d', 'ebitda_mev', 'f_score',
    'inv_gr1', 'kz_index', 'ni_me', 'o_score', 'prc_highprc_252d',
    'qmj', 'qmj_growth', 'resff3_6_1', 'ret_12_1', 'ret_3_1', 'ret_6_1',
    'sale_me', 'z_score'
]

# 'date'列も残すように先頭に追加（列名が date の場合）
columns_to_keep = ['date'] + selected_factors

# 存在する列のみ抽出（安全策）
columns_existing = [col for col in columns_to_keep if col in jkp_corr_filtered.columns]
jkp_selected = jkp_corr_filtered[columns_existing]
jkp_selected = jkp_selected.reset_index()

The number of dropped factors due to high missing ratio: 0
The dropped factors due to low variance: 0
The number of dropped factors due to high correlation: 43


(240, 19)

### Compustat Processing

In [23]:
df = pd.read_csv("CompFirmCharac.csv")

# 日付列の変換（例: 'datadate'列がある場合）
df['datadate'] = pd.to_datetime(df['datadate'], errors='coerce')

sp500_firms = [1013, 1075, 1078, 1161, 1177, 1209, 1239, 1240, 1279, 1300, 1318, 1356, 1380, 1408, 1440, 1447, 1449, 1478, 1487, 1567, 1602, 1619, 1632, 1651, 1661, 1663, 1678, 1690, 1704, 1722, 1794, 1878, 1891, 1913, 1920, 1976, 1988, 1995, 2019, 2044, 2085, 2086, 2111, 2136, 2146, 2154, 2184, 2230, 2255, 2269, 2285, 2290, 2312, 2403, 2435, 2444, 2490, 2504, 2547, 2574, 2663, 2710, 2751, 2783, 2817, 2845, 2849, 2884, 2968, 2991, 3011, 3024, 3054, 3062, 3105, 3121, 3144, 3170, 3221, 3226, 3231, 3243, 3278, 3310, 3336, 3362, 3413, 3439, 3497, 3502, 3505, 3532, 3555, 3650, 3734, 3735, 3813, 3835, 3897, 3964, 3980, 4016, 4029, 4040, 4058, 4060, 4062, 4066, 4087, 4093, 4145, 4194, 4199, 4213, 4242, 4321, 4367, 4371, 4423, 4503, 4517, 4560, 4598, 4601, 4611, 4640, 4674, 4699, 4723, 4737, 4739, 4818, 4839, 4843, 4885, 4988, 4990, 5046, 5047, 5071, 5073, 5074, 5125, 5216, 5229, 5234, 5250, 5256, 5439, 5518, 5568, 5589, 5597, 5606, 5643, 5680, 5723, 5742, 5786, 5860, 5878, 5959, 6008, 6066, 6078, 6081, 6097, 6104, 6136, 6241, 6266, 6268, 6304, 6307, 6310, 6375, 6403, 6435, 6475, 6502, 6529, 6649, 6669, 6730, 6733, 6742, 6768, 6774, 6781, 6799, 6821, 6829, 7017, 7065, 7067, 7085, 7116, 7139, 7146, 7154, 7163, 7171, 7186, 7228, 7238, 7241, 7257, 7260, 7267, 7343, 7366, 7409, 7435, 7506, 7585, 7620, 7637, 7647, 7648, 7711, 7750, 7772, 7866, 7875, 7881, 7904, 7906, 7922, 7923, 7974, 7977, 7982, 7985, 8007, 8030, 8068, 8099, 8245, 8247, 8253, 8264, 8272, 8304, 8358, 8402, 8446, 8455, 8470, 8479, 8488, 8530, 8536, 8539, 8543, 8549, 8606, 8762, 8787, 8810, 8823, 8972, 9004, 9063, 9203, 9217, 9258, 9299, 9351, 9359, 9372, 9411, 9459, 9465, 9483, 9555, 9667, 9699, 9778, 9783, 9846, 9850, 9882, 9899, 10016, 10035, 10096, 10115, 10121, 10156, 10187, 10190, 10232, 10247, 10277, 10332, 10391, 10405, 10420, 10426, 10453, 10499, 10507, 10519, 10530, 10614, 10726, 10787, 10793, 10860, 10867, 10903, 10920, 10974, 10983, 10984, 11060, 11220, 11228, 11259, 11264, 11366, 11446, 11456, 11465, 11506, 11609, 11636, 11672, 11687, 11818, 11856, 11910, 11923, 11925, 12053, 12123, 12124, 12136, 12138, 12141, 12142, 12216, 12233, 12338, 12389, 12540, 12635, 12689, 12726, 12756, 12884, 12886, 13041, 13204, 13341, 13498, 13561, 13646, 13714, 13988, 14256, 14324, 14477, 14489, 14535, 14565, 14590, 14623, 14624, 14650, 14824, 14934, 14960, 15084, 15202, 15208, 15247, 15521, 15708, 15709, 15855, 16243, 16478, 16721, 17110, 17130, 18699, 20423, 20779, 22140, 22260, 22325, 22794, 23592, 23809, 23877, 23943, 23978, 24008, 24032, 24216, 24287, 24318, 24379, 24468, 24607, 24800, 24856, 25056, 25124, 25157, 25279, 25283, 25338, 25340, 25356, 25434, 25495, 25880, 26011, 27845, 27914, 27928, 28034, 28139, 28195, 28256, 28338, 28349, 28733, 28930, 29028, 29095, 29241, 29345, 29389, 29392, 29791, 29984, 30128, 30490, 30697, 30865, 30990, 31122, 31166, 31549, 31673, 31846, 61489, 61552, 61574, 61591, 61676, 61739, 62599, 62634, 62689, 63180, 63501, 63669, 63766, 63800, 63892, 64156, 64166, 64356, 64768, 65048, 65417, 65904, 66708, 66731, 110685, 112033, 113419, 114524, 114628, 117768, 119417, 120877, 125533, 126554, 133768, 135484, 136648, 137232, 139662, 139665, 140541, 140760, 143356, 144066, 144559, 145046, 145701, 149738, 157858, 158354, 160255, 164708]
sp500_firms_str = [str(x) for x in sp500_firms]
df['gvkey'] = df['gvkey'].astype(str)
df_500 = df[df['gvkey'].isin(sp500_firms_str)]

df_500 = df_500[df_500['datadate'].dt.year >= 2005]

end_date = pd.to_datetime("2024-09-30")
df_500['datadate'] = pd.to_datetime(df_500['datadate'], errors='coerce')
latest_dates = df_500.groupby('gvkey')['datadate'].max()
valid_gvkeys = latest_dates[latest_dates >= end_date].index
df_upto_2024 = df_500[df_500['gvkey'].isin(valid_gvkeys)].copy()

cutoff_date = pd.to_datetime("2024-9-30")
df_500 = df_500[df_500['datadate'] <= cutoff_date].copy()

df_500['date'] = df_500['datadate'] + pd.offsets.MonthEnd(0)

id_column = 'gvkey' if 'gvkey' in df_500.columns else 'cik'
comp_df_cleaned = df_500.dropna(subset=['date', id_column])
comp_df_cleaned = comp_df_cleaned.sort_values(by=['date', id_column])

numeric_cols = comp_df_cleaned.select_dtypes(include=['int64', 'float64']).columns

with pd.option_context('display.max_rows', None):
    print(comp_df_cleaned[numeric_cols].notnull().sum().sort_values())

with pd.option_context('display.max_rows', None):
    print(comp_df_cleaned[numeric_cols].nunique().sort_values())

#na < 15000 (half of the total rows)
na_columns = [
    'fuseoy', 'unwccy', 'utfdocy', 'utfoscy', 'uwkcapcy', 'wcapchy', 'wcapcy', 'ufretsdy', 'ustdncy',
    'tsafcy', 'fsrcty', 'fusety', 'fsrcoy', 'fopty', 'arcedy', 'arceepsy', 'arcey', 'gdwlamy', 'utmey',
    'amcy', 'ffoy', 'srety', 'nimy', 'dprety', 'itccy', 'udfccy', 'finxopry', 'finxinty', 'afudciy',
    'afudccy', 'finrevy', 'pdvcy', 'esuby', 'plly', 'niity', 'tiey', 'tiiy', 'ncoy', 'usubdvpy',
    'gdwliepsy', 'gdwlidy', 'prstkccy', 'gdwliay', 'rrdy', 'rrepsy', 'gdwlipy', 'rray', 'rrpy',
    'tdcy', 'uspiy', 'prstkpcy', 'scstkcy', 'udvpy', 'uoisy', 'depcy', 'uaolochy', 'ugiy', 'spstkcy',
    'unopincy', 'uniamiy', 'updvpy', 'cdvcy', 'uptacy', 'wdepsy', 'wddy', 'wday', 'wdpy', 'glivy',
    'glepsy', 'gldy', 'glay', 'glpy', 'dteepsy', 'dtedy', 'dteay', 'dtepy', 'derhedgly', 'hedgegly',
    'setepsy', 'setdy', 'setay', 'setpy', 'nrtxtepsy', 'nrtxtdy', 'nrtxty', 'spiepsy', 'spidy',
    'spioay', 'spiopy', 'txdiy', 'fcay', 'aqepsy', 'aqdy', 'aqay', 'aqpy', 'optfvgry', 'glceepsy',
    'glcedy', 'glceay', 'glcepy', 'optdry', 'optlifey', 'optrfry', 'optvoly', 'stkcpay', 'pncidy',
    'pnciepsy', 'pncwidy', 'pncwiepsy', 'pnciay', 'pncipy', 'pncwiay', 'pncwipy', 'tfvcey', 'xrdy',
    'pncidpy', 'pnciepspy', 'pncwiepy', 'pncwidpy', 'pnciapy', 'pncippy', 'pncwippy', 'pncwiapy',
    'rcepsy', 'rcdy', 'rcay', 'spcedy', 'spceepsy', 'rcpy', 'spcedpy', 'spceepspy', 'spcey', 'spcepy'
]

#unique counts < 2
constant_columns = [
    'fopty', 'uwkcapcy', 'wcapchy', 'wcapcy', 'ustdncy', 'unwccy',
    'utfdocy', 'tsafcy', 'ufretsdy', 'fusety', 'fuseoy', 'fsrcty',
    'fsrcoy', 'utfoscy', 'gdwlamy', 'updvpy'
]

selected_columns = list(set(numeric_cols) - set(na_columns) - set(constant_columns))

cols = ['gvkey','datadate'] + selected_columns
df_num = comp_df_cleaned[cols]
df_num = df_num.set_index(['gvkey','datadate'])

for col in selected_columns:
    if col in df_num.columns and pd.api.types.is_numeric_dtype(df_num[col]):
        nan_mask = df_num[col].isnull()
        valid_series = df_num[col].dropna()
        
        winsorized_array = winsorize(valid_series.to_numpy(), limits=[0.01, 0.01])
        df_num.loc[~nan_mask, col] = pd.Series(winsorized_array, index=valid_series.index)

scaler = StandardScaler()
df_scaled = df_num.copy()

# 選択された数値列に標準化を適用
df_scaled[selected_columns] = scaler.fit_transform(df_num[selected_columns])

selected_columns = [
    col for col in selected_columns
    if col not in ['rdipay', 'acchgy', 'rdipy', 'rdipdy', 'rdipepsy', 'txwy']
]
correlation_matrix = df_scaled[selected_columns].corr()
highly_correlated_pairs = []
temp_highly_correlated_vars_set = set() 

for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        correlation_value = correlation_matrix.iloc[i, j]
        
        if abs(correlation_value) >= 0.7:
            highly_correlated_pairs.append(((col1, col2), correlation_value))
            temp_highly_correlated_vars_set.add(col1)
            temp_highly_correlated_vars_set.add(col2)

# List of highly correlated values
highly_correlated_vals = list(temp_highly_correlated_vars_set)
duplicated_var = [
    col for col in highly_correlated_vals 
    if col not in ['xoptdy','xinty','niy','revty','epspiy','cshpry','dpy']
]

selected_columns = [
    col for col in selected_columns
    if col not in duplicated_var
]
len(selected_columns)
display(selected_columns)


columns_to_extract = [
    'datadate','gvkey','cusip','fqtr', 'cstkey', 'cicurry', 'dilady', 'niy', 'nopiy', 'invchy', 'esubcy', 'apalchy', 'aolochy',
    'exrey', 'spiy', 'aqcy', 'ivstchy', 'txbcoy', 'fyr', 'recchy', 'txbcofy', 'cisecgly', 'xoptdy',
    'xidocy', 'xiy', 'fiaoy', 'cipeny', 'revty', 'txdcy', 'xinty', 'ivncfy', 'sstky', 'dlcchy',
    'fyearq', 'cidergly', 'exchg', 'ivacoy', 'fincfy', 'cik', 'epspiy', 'dpy', 'sppivy', 'chechy',
    'ciothery', 'prstkcy', 'cshpry', 'txachy', 'sppey'
]

# 指定された列だけを抽出（存在しない列は無視する）
df_selected = comp_df_cleaned[[col for col in columns_to_extract if col in df_500.columns]]
df_selected['datadate'] = pd.to_datetime(df_selected['datadate'], errors='coerce')


expanded_rows = []

for _, row in df_selected.iterrows():
    base_date = pd.to_datetime(row['datadate'])
    for offset in range(1, 4):  # t+1, t+2, t+3
        date = (base_date + pd.DateOffset(months=offset)).replace(day=1) + pd.offsets.MonthEnd(0)
        new_row = row.copy()
        new_row['date'] = date
        expanded_rows.append(new_row)

# 拡張されたデータフレームを作成
df_expanded = pd.DataFrame(expanded_rows)

comp_selected = df_expanded.sort_values(by=['date'])
comp_selected.shape

/var/folders/_s/jd19jm5x0bj6k6nfh5q_1n700000gn/T/ipykernel_41671/2423101258.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("CompFirmCharac.csv")


fuseoy            0
wcapcy            0
utfoscy           0
utfdocy           0
ustdncy           0
arcey             0
arceepsy          0
arcedy            0
fsrcty            0
unwccy            0
wcapchy           0
uwkcapcy          0
tsafcy            0
fusety            0
fopty             0
fsrcoy            0
ufretsdy          0
gdwlamy           1
utmey           114
amcy            121
ffoy            377
srety           417
nimy            435
dprety          494
itccy           506
udfccy          716
finxopry        727
finxinty        829
afudciy         850
afudccy         850
finrevy         994
pdvcy          1244
esuby          1561
plly           1614
niity          1658
ncoy           1705
tiiy           1705
tiey           1705
usubdvpy       1720
gdwliepsy      1913
gdwlidy        1916
prstkccy       1979
gdwliay        1992
gdwlipy        2010
rrepsy         2034
rrdy           2034
tdcy           2067
rray           2071
rrpy           2084
uspiy          2101


['spiy',
 'dpy',
 'dlcchy',
 'ciothery',
 'fiaoy',
 'recchy',
 'xoptdy',
 'sppivy',
 'ivstchy',
 'xinty',
 'fyearq',
 'ivncfy',
 'prstkcy',
 'esubcy',
 'cstkey',
 'txachy',
 'cipeny',
 'sstky',
 'cshpry',
 'revty',
 'cicurry',
 'exrey',
 'epspiy',
 'fyr',
 'txdcy',
 'xidocy',
 'apalchy',
 'exchg',
 'aolochy',
 'invchy',
 'txbcoy',
 'aqcy',
 'xiy',
 'cisecgly',
 'dilady',
 'niy',
 'cidergly',
 'txbcofy',
 'chechy',
 'fincfy',
 'ivacoy',
 'cik',
 'fqtr',
 'sppey',
 'nopiy']

/var/folders/_s/jd19jm5x0bj6k6nfh5q_1n700000gn/T/ipykernel_41671/2423101258.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['datadate'] = pd.to_datetime(df_selected['datadate'], errors='coerce')


(94047, 49)

### Merge JKP with Compustat

In [27]:
comp = comp_selected
jkp = jkp_selected

jkp['date'] = pd.to_datetime(jkp['date'], errors='coerce')
comp['date'] = pd.to_datetime(comp['date'], errors='coerce')

# 3. マージ（左側はcompustat、右側にファクター）
merged = comp.merge(jkp, on='date', how='left')

non_feature_cols = ['date', 'gvkey', 'cik']
feature_cols = [col for col in merged.columns if col not in non_feature_cols and pd.api.types.is_numeric_dtype(merged[col])]

# 1. 欠損値を中央値で補完
merged[feature_cols] = merged[feature_cols].fillna(merged[feature_cols].median())

# 2. 標準化
scaler = StandardScaler()
merged_scaled = merged.copy()
merged_scaled[feature_cols] = scaler.fit_transform(merged[feature_cols])

# 3. 相関行列
corr_matrix = merged_scaled[feature_cols].corr().abs()

# 4. 上三角行列の抽出（NumPyを使う）
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 5. 相関が高い列のうち片方を削除
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# 6. 最終データ（MLP用）
merged_jkp_comp = merged_scaled.drop(columns=to_drop)

gvkey_cusip_counts = merged_jkp_comp.groupby('gvkey')['cusip'].nunique()

# 2. 複数の cusip を持つ gvkey を抽出
multi_cusip_gvkeys = gvkey_cusip_counts[gvkey_cusip_counts > 1].index

# 3. 対象企業のデータを抽出
merged_multi_cusip = merged_jkp_comp[merged_jkp_comp['gvkey'].isin(multi_cusip_gvkeys)]

print(merged_multi_cusip.head())

merged_jkp_comp['cusip8'] = merged_jkp_comp['cusip'].str[:8]
unique_cusip8_count = merged_jkp_comp['cusip8'].nunique()
print(f"Number of unique cusip8: {unique_cusip8_count}")

Empty DataFrame
Columns: [datadate, gvkey, cusip, fqtr, cstkey, cicurry, dilady, niy, nopiy, invchy, esubcy, apalchy, aolochy, exrey, spiy, aqcy, ivstchy, txbcoy, fyr, recchy, txbcofy, cisecgly, xoptdy, xidocy, xiy, fiaoy, cipeny, revty, txdcy, xinty, ivncfy, sstky, dlcchy, fyearq, cidergly, exchg, ivacoy, fincfy, cik, epspiy, dpy, sppivy, chechy, ciothery, prstkcy, cshpry, txachy, sppey, date, beta_60m, bidaskhl_21d, dolvol_var_126d, ebitda_mev, f_score, inv_gr1, kz_index, ni_me, o_score, prc_highprc_252d, qmj, qmj_growth, resff3_6_1, ret_12_1, ret_3_1, ret_6_1, sale_me, z_score]
Index: []

[0 rows x 67 columns]
Number of unique cusip8: 473


### Merge JKP & Compustat data with Monthly CRSP

In [29]:
crsp = pd.read_csv("monthly_crsp.csv")

crsp['date'] = pd.to_datetime(crsp['MthCalDt'], errors='coerce')

# 2005年以降にフィルタリング
start_date = pd.to_datetime("2005-01-01")
end_date = pd.to_datetime("2024-12-31")

# フィルタ処理
crsp_filtered = crsp[(crsp['date'] >= start_date) & (crsp['date'] <= end_date)].copy()

cusip_set = merged_jkp_comp['cusip8'].dropna().unique()
crsp_matched = crsp[crsp['CUSIP'].isin(cusip_set)]
unique_cusip8_crsp = crsp_matched['CUSIP'].nunique()
print(f"Number of unique CUSIP8 in CRSP: {unique_cusip8_crsp}")
merged_jkp_comp['date'] = pd.to_datetime(merged_jkp_comp['date'], errors='coerce')
crsp_matched['date'] = pd.to_datetime(crsp_matched['date'], errors='coerce')
crsp_matched['cusip8'] = crsp_matched['CUSIP'].str[:8]


merged_data = pd.merge(
    merged_jkp_comp,
    crsp_matched,
    left_on=['date', 'cusip8'],
    right_on=['date', 'cusip8'],
    how='inner'  # 必要に応じて 'left' や 'outer' に変更可能
)

merged_data.columns.tolist()
merged_data.to_csv("merged_compustat_crsp.csv", index=False)

Number of unique CUSIP8 in CRSP: 457


/var/folders/_s/jd19jm5x0bj6k6nfh5q_1n700000gn/T/ipykernel_41671/3754352724.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_matched['date'] = pd.to_datetime(crsp_matched['date'], errors='coerce')
/var/folders/_s/jd19jm5x0bj6k6nfh5q_1n700000gn/T/ipykernel_41671/3754352724.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_matched['cusip8'] = crsp_matched['CUSIP'].str[:8]
